In [1]:
import sqlite3
import json
import pandas as pd
import ollama
from pathlib import Path

db_path = Path(r"C:\Users\cassi\OneDrive\Documents\ESILV_A5S1_V2\PLM\Hackathon\HACKATHON_The_regulatory-_GPS\Data_ingestion_and_sources\Data_extracted\France.db")

if not db_path.exists():
    raise FileNotFoundError(f"Le fichier n'existe pas : {db_path}")

country = db_path.stem.upper()  # "India" → "INDIA"

conn = sqlite3.connect(str(db_path))

# Vérifier les tables disponibles
tables = pd.read_sql_query(
    "SELECT name FROM sqlite_master WHERE type='table'", conn
)
print("Tables disponibles :", tables)

# Charger la table principale (adapter si besoin)
df = pd.read_sql_query("SELECT * FROM regulations", conn)

print(f"Nombre d’entrées chargées : {len(df)}")
print(f"Pays détecté : {country}")


Tables disponibles :               name
0      regulations
1  sqlite_sequence
Nombre d’entrées chargées : 5
Pays détecté : FRANCE


In [2]:
def build_prompt(content, country):
    return f"""
Tu es un extracteur de réglementation. Tu dois retourner STRICTEMENT du JSON valide.

NE PRODUIS :
- AUCUN commentaire
- AUCUN texte descriptif
- AUCUNE explication
- AUCUN retour à la ligne en dehors du JSON
- AUCUN ```json``` ou markdown

Tu dois répondre EXCLUSIVEMENT avec un JSON valide respectant strictement la structure suivante :

{{
  "pays": "{country}",
  "nom_norme": "string ou 'inconnu'",
  "date_publication": "string ou 'inconnu'",
  "loi_id": "string ou 'inconnu'",
  "pieces_concernees": ["liste de strings"],
  "exigences": [
    {{
      "description": "string",
      "obligation": "obligatoire/interdit/recommandé/inconnu",
      "tests_requis": ["liste ou vide"],
      "seuils": ["liste ou vide"],
      "article_source": "string ou inconnu"
    }}
  ]
}}

Si une information n’existe pas, mets "inconnu".
- Les réglementations étant parfois complexes, fais de ton mieux pour extraire les informations importantes et compréhensibles.
- L'idée n'est pas d'avoir des textes juridiques mais des contraintes claires et exploitables.

Voici le texte réglementaire à analyser :
\"\"\"
{content}
\"\"\"

RÉPONDS UNIQUEMENT PAR UN JSON VALIDE.
"""

In [3]:
# ------------------------------------------------------
# 3. Fonction qui interroge LLaMA via Ollama
# ------------------------------------------------------

def run_llama(prompt):
    response = ollama.generate(
        model="llama3.1",
        prompt=prompt,
        options={"temperature": 0.1}  # très faible pour éviter les hallucinations
    )
    return response["response"]

In [ ]:
# ------------------------------------------------------
# 4. Boucle sur les lignes de la DB
# ------------------------------------------------------

results = []

for idx, row in df.iterrows():
    content = row["content"]
    prompt = build_prompt(content, country)
    
    print(f"\nProcessing item {idx}...")

    raw_output = run_llama(prompt)

    try:
        json_obj = json.loads(raw_output)
    except json.JSONDecodeError:
        print("⚠ JSON invalide, tentative de correction…")
        # Petit fallback : essayer de récupérer la partie JSON
        try:
            start = raw_output.index("{")
            end = raw_output.rindex("}") + 1
            json_obj = json.loads(raw_output[start:end])
        except:
            print("❌ Impossible de parser le JSON.")
            continue
    
    results.append(json_obj)



Processing item 0...
⚠ JSON invalide, tentative de correction…

Processing item 1...


In [ ]:
# ------------------------------------------------------
# 5. Export final
# ------------------------------------------------------

output_path = f"structured_{country.lower()}.json"
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(results, f, ensure_ascii=False, indent=2)

print(f"\n✔ Export terminé → {output_path}")